# 한국복지패널 데이터 분석
- 한국보건사회연구원에서 우리나라 가구의 경제활동을 연구한 데이터
- 전국 7,000여 가구를 선정하여 2006년 부터 매년 추적 조사한 자료
- 경제활동, 생활실태, 복지욕구 등 천여개 변수로 구성됨
- 통계 패키지 SPSS 전용 파일

### Lab 1: 데이터 분석 준비하기
1. 데이터 준비
- Koweps_hpwc14_2019_beta2.sav
- 2020년 발간된 복지패널 데이터, 6331가구, 14,418명 정보를 담고 있음
2. 패키지 설치 및 로드
- 데이터 파일은 통계 분석 패키지인 SPSS 전용 파일
- pyreadstat 패키지 설치하면 SPSS, SAS, STATA등 다양한 통계 분석 소프트웨어 데이터 파일을 불러올 수 있음
- pip install pyreadstat

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#pyreadstat 패키지 설치
# !pip install pyreadstat

# pyreadstat 패키지의 설치 정보 확인
!pip show pyreadstat

Name: pyreadstat
Version: 1.2.4
Summary: Reads and Writes SAS, SPSS and Stata files into/from pandas data frames.
Home-page: https://github.com/Roche/pyreadstat
Author: Otto Fajardo
Author-email: pleasecontactviagithub@notvalid.com
License: Apache License Version 2.0
Location: D:\anaconda3\Lib\site-packages
Requires: pandas
Required-by: 


3. 데이터 불러오기
- pd.read_spss()로 데이터 불러오기
- df.copy()로 복사본을 만들어 분석에 활용한다.

In [ ]:
# SPSS 통계 패키지 전용 파일 데이터 불러오기
# SPSS(Statistical Package for the Social Sciences)는 사회 과학용 통계 패키지로
# IBM
raw_welfare = pd.read_spss('Koweps_hpwc14_2019_beta2.sav')
#데이터 분석 작업용 복사본 만들기
welfare = raw_welfare.copy()

4. 데이터 검토하기
- 데이터의 특성을 살펴보기
- 행, 열 개수
- 변수 속성
- 요약 통계량

In [ ]:
# 데이터 구조 특징 살펴보기
# 변수의 수가 많고 변수명이 코드로 되어 있어서 전체 구조 파악이 힘들다.
welfare.head()

In [ ]:
# 행과 열 개수 출력
welfare.shape

In [ ]:
# 요약 통계량
welfare.describe()

5. 변수명 바꾸기
- 복지데이터와 같은 대규모 데이터는 변수의 수가 많고 변수명이 코드로 되어 있다.
- 규모가 큰 조사 자료의 경우 데이터 특징을 설명한 코드북을 함께 제공한다.
- 코드북에는 변수명과 값의 의미가 설명되어 있다. (코드북을 통해 데이터 특징과 분석 방향 아이디어 도출)
- 복지 데이터 코드북 일부 : Koweps_Codebook_2019.xlsx
- 7개 변수를 선정하여 분석한다.
    - 1단계 : 변수 검토 및 전처리 - 변수 특징을 파악하고, 이상치와 결측치 정제
    - 2단계 : 변수 간 관계 분석 - 데이터 요약 표와 특징을 알 수 있는 그래프를 만들고 해석한다

In [ ]:
# 코드북을 참조하여 분석에 사용할 7개 변수명을 알기쉬운 단어로 변경
welfare = welfare.rename(
columns = {
    'h14_g3':'sex', # 성별
    'h14_g4':'birth', # 태어난 연도
    'h14_g10':'marriage_type', # 혼인 상태
    'h14_g11':'religion', # 종교
    'p1402_8aq1':'income', # 일한달의 월 평균 임금
    'h14_eco9':'code_job', # 직종
    'h14_reg7':'code_region' # 7개 권역별 지역 구분
})

In [ ]:
# 바뀐 변수명으로 데이터 확인
welfare[['sex','birth','marriage_type','religion','income','code_job','code_region']].head()

## 성별에 따른 월급 차이 - 성별에 따라 월급에 차이가 있을까?
### Lab 2: 성별 변수 검토 및 전처리하기

1. 변수 검토하기

In [ ]:
# 성별로 몇명이 있는지 확인
# 성별 변수의 데이터 타입 확인
welfare['sex'].dtypes

In [ ]:
type(welfare['sex'])

In [ ]:
#성별 분류별 빈도 수 구하기 (성별은 1, 2로 구성)
welfare['sex'].value_counts()

- 성별 값에 대한 코드북 확인
- 여자 7913명, 남자 6505명
- 모름/무응답(9) 값은 없는 것으로 확인된다.
- 남자 1, 여자 2, 모름/무응답 9

2. 전처리 하기
- 남자(1), 여자(2) 값 이외에 모름/무응답(9)이나 기타 이상차가 있는 경우 결측 처리
- sex 변수의 값 1, 2에 대해 값의 의미를 알기 쉽도록 'male','female'로 변경

In [ ]:
#이상치에 대한 결측치 처리
welfare['sex'] = np.where(welfare['sex']==9,np.nan,welfare['sex'])
#결측치 확인
welfare['sex'].isna().sum()

In [ ]:
#성별 항목 이름 부여
welfare['sex']=np.where(welfare['sex']==1,'male','female')
welfare['sex'].value_counts()

In [ ]:
#성별 빈도수에 따른 막대 그래프 그리기
sns.countplot(data=welfare,x='sex')

#### countplot을 사용하지 않고, barplot을 사용해서 그리려면?

In [ ]:
result = welfare.groupby('sex',as_index=False).agg(n=('sex','count'))
result

In [ ]:
sns.barplot(result,x='sex',y='n')

### Lab 3: 월급 변수 검토 및 전처리하기
1. 변수 검토하기
- 코드북: 월급은 '일한 달의 월 평균 임금'으로 1만원 단위 기록
월급은 연속 변수로 그룹별 빈도수(df.value_counts())로 파악하기 어려움
연속 변수는 df.describe()로 요약 통계량으로 파악하는 것이 바람직

In [ ]:
#월급 변수의 데이터 타입 확인
welfare['income'].dtypes

In [ ]:
#월급의 요약 통계량 구하기
welfare['income'].describe()

월급 요약 통계량 분석
- 0~1892만원 분포
- 150만원 ~ 345만원에 가장 많이 분포
- 평균 값은 268.445만원
- 중앙 값은 220만원(전반적으로 낮은 값 쪽으로 치우쳐 있음)

In [ ]:
# 월급 분포를 히스토그램으로 확인
sns.histplot(data=welfare,x='income')

In [ ]:
sns.histplot(data=welfare,x='income',bins=10)

0~250만원 대에 가장 많은 사람이 분포하고, 그 뒤로 점차 빈도가 감소

2. 전처리 하기
- 코드북을 보면, 원급은 만원 단위로 되어 있고
- 모름/무응답은 9999로 코딩됨

        - 이상치나 결측치 확인
            - 월급의 값이 9999인 경우, 이상치로 판단하여 결측 처리
            - 결측치 확인

In [ ]:
# 이상치 확인
welfare['income'].describe()

In [ ]:
welfare['income'].head()

- 이상치 확인: 월급 최대치가 1892만원.
- 모름/무응답 (9999)는 없는 것으로 확인

In [ ]:
# 결측치 확인
# 직업이 없어서 월급을 받지 못하는 응답자
welfare['income'].isna().sum()

In [ ]:
# 이상치(9999) 결측 처리
welfare['income'] = np.where(welfare['income']==9999,np.nan,welfare['income'])

In [ ]:
# 결측치 제거
# welfare.dropna(subset=['income'])['income'].isna().sum()

### Lab 4: 성별에 따른 월급 차이 분석하기
1. 성별 원급 평균표 만들기

In [ ]:
# 성별 월급 평균표 만들기
sex_income = welfare.dropna(subset = ['income'])\
                    .groupby('sex',as_index=False)\
                    .agg(mean_income = ('income','mean'))
sex_income

In [ ]:
# 그래프 만들기
sns.barplot(data=sex_income,x='sex',y='mean_income')